In [2]:
#### lib required for dashboard
import plotly.graph_objects as go
import dash
import dash_html_components as html
import dash_core_components as dcc

import plotly.offline as pyo

from dash.dependencies import State, Input, Output

import numpy as np
import pandas as pd

In [3]:
d1=pd.read_csv("data.csv");

In [4]:
def datagen(data_r):
    cat_g=sorted(data_r['ID'].unique())
    data =pd.DataFrame(columns=data_r.columns)
    sampled_data =pd.DataFrame(columns=data.columns)
    for i in cat_g:
        grouped_data=data_r[data_r['ID']==i].groupby('Time').mean()
        grouped_data=grouped_data.reset_index()
        sampled_data=pd.concat([sampled_data,grouped_data],ignore_index= True, sort=False)
    return(sampled_data)

In [5]:
cat_g=sorted(d1['ID'].unique())

## Creating values and lables for dropdown

options_list = []
for i in cat_g:
    options_list.append({'label': i, 'value': i})

In [ ]:
def fig_generator(sample_data,input_cat):
    plot_data =[]
    
    temp=['T1','T2','T3','Tamb']
    for j in temp:
        plot_data.append(go.Scatter(x=sample_data['Time'], y=sample_data[j], name = j ))
    plot_layout = go.Layout(title = "Temperature Reading for Device "+str(input_cat))

    fig = go.Figure( data = plot_data ,layout = plot_layout)

    return(fig.data,fig.layout)


In [ ]:
app = dash.Dash()

### defining the HTML component


app.layout = html.Div(children=[html.Div("SENSOR TEMPRATURE DASHBOAD",style= {   "color": "white",
                                                      "text-align": "center","background-color": "blue",
                                                      "border-style": "dotted","display":"inline-block","width":"90%"
                                                      
                                                    }),
                       html.Div(dcc.Dropdown(id = "drop_down_1" ,options= options_list , value= 1
                                                       ),style= {
                                                      "color": "green",
                                                      "text-align": "center","background-color": "darkorange",
                                                      "border-style": "dotted","display":"inline-block","width":"20%",
                                                      
                                                    }),
                       html.Div(children=[html.P(
                            id="map-title",
                            children = "DISPLAY BOARD ",
                        ), html.Div(dcc.Graph(id ="plot_area"))
                                                       ],style= {
                                                      "color": "black",
                                                      "text-align": "center","background-color": "yellow",
                                                      "border-style": "dotted","display":"inline-block","width":"70%",
                                                                                                            
                                                    })],style={"width":"100%",'paffing':10})

## Creating callback buttons

@app.callback(Output("plot_area", 'figure'),
              
              [Input("drop_down_1", "value")])

def updateplot(input_cat):
    
    df= datagen(d1)
    sample_data = df[df["ID"] ==input_cat]
    
    trace,layout = fig_generator(sample_data,input_cat)
    
    return {
        'data': trace,
        'layout':layout
    }
    
    
    
if __name__=='__main__':
    app.run_server()
	
	